In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from time import strftime, localtime
TCT = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Company_Tweet.csv\Company_Tweet.csv", index_col="tweet_id")
TT  = pd.read_csv(r"C:\Users\Fabio\OneDrive\Documents\Studies\Final Project\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\twitter data\Tweets about the Top Companies from 2015 to 2020\Tweet.csv\Tweet.csv", index_col="tweet_id")

In [2]:
TT["body"]

tweet_id
550441509175443456     lx21 made $10,008  on $AAPL -Check it out! htt...
550441672312512512     Insanity of today weirdo massive selling. $aap...
550441732014223360     S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...
550442977802207232     $GM $TSLA: Volkswagen Pushes 2014 Record Recal...
550443807834402816     Swing Trading: Up To 8.91% Return In 14 Days h...
                                             ...                        
1212159765914079234    That $SPY $SPX puuump in the last hour was the...
1212159838882533376    In 2020 I may start Tweeting out positive news...
1212160015332728833    Patiently Waiting for the no twitter sitter tw...
1212160410692046849    I don't discriminate. I own both $aapl and $ms...
1212160477159206912    $AAPL #patent 10,522,475 Vertical interconnect...
Name: body, Length: 3717964, dtype: object

In [3]:
# Load the regular expression library
import re
TT = TT[:100]
# Remove punctuation
TT['body_processed'] = TT['body'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
TT['body_processed'] = TT['body_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
TT['body_processed'].head()


tweet_id
550441509175443456    lx21 made $10008  on $aapl -check it out http:...
550441672312512512    insanity of today weirdo massive selling $aapl...
550441732014223360    s&p100 #stocks performance $hd $low $sbux $tgt...
550442977802207232    $gm $tsla: volkswagen pushes 2014 record recal...
550443807834402816    swing trading: up to 891% return in 14 days ht...
Name: body_processed, dtype: object

In [4]:
# Import the wordcloud library
import time
from wordcloud import WordCloud
time.strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# Join the different processed titles together.
long_string = ','.join(list(TT['body_processed'].values))
time.strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=1000, contour_width=3, contour_color='steelblue')
time.strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# Generate a word cloud
wordcloud.generate(long_string)
time.strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# Visualize the word cloud
#wordcloud.to_image()

'2012-09-13 08:22:50'

In [5]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]


data = TT['body_processed'].values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)

print(data_words[:1][0][:30])

['lx', 'made', 'aapl', 'check', 'http', 'profitly', 'mnd', 'saff', 'learn', 'howtotrade', 'http', 'bitly', 'nljx', 'exe', 'watt', 'imrs', 'cach', 'gmo']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fabio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)]


In [7]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.027*"aapl" + 0.025*"google" + 0.024*"year" + 0.021*"apple" + 0.016*"http" '
  '+ 0.016*"review" + 0.016*"googl" + 0.016*"part" + 0.016*"pay" + '
  '0.016*"end"'),
 (1,
  '0.065*"http" + 0.026*"amzn" + 0.020*"goog" + 0.019*"richest" + '
  '0.018*"people" + 0.018*"world" + 0.018*"add" + 0.017*"amazon" + '
  '0.015*"bitly" + 0.015*"stock"'),
 (2,
  '0.040*"http" + 0.037*"amazon" + 0.037*"amzn" + 0.036*"onwsjcom" + '
  '0.033*"year" + 0.033*"worst" + 0.032*"since" + 0.032*"billion" + '
  '0.032*"lost" + 0.032*"bezos"'),
 (3,
  '0.057*"http" + 0.033*"aapl" + 0.025*"amzn" + 0.016*"world" + 0.013*"stksco" '
  '+ 0.013*"people" + 0.013*"richest" + 0.013*"add" + 0.012*"year" + '
  '0.011*"top"'),
 (4,
  '0.041*"http" + 0.028*"amzn" + 0.023*"aapl" + 0.017*"highest" + '
  '0.015*"earlier" + 0.015*"caused" + 0.015*"drop" + 0.015*"suddenly" + '
  '0.015*"wafkrs" + 0.015*"glitch"'),
 (5,
  '0.022*"buy" + 0.015*"valued" + 0.015*"sell" + 0.015*"goog" + '
  '0.015*"roughly" + 0.015*"fairly" +

In [8]:
import os
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle 


# Visualize the topics
pyLDAvis.enable_notebook()

pyLDAvis_path = "C:\\Users\\Fabio\\OneDrive\\Documents\\Studies\\Final Project\\Social-Media-and-News-Article-Sentiment-Analysis-for-Stock-Market-Autotrading\\pyLDAvis\\"

#LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
LDAvis_data_filepath = os.path.join(pyLDAvis_path, 'ldavis_prepared_'+str(num_topics))


# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, 'ldavis_prepared_'+ str(num_topics) +'.html')

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.197355 -0.003940       1        1  18.800984
7     -0.068458 -0.184085       2        1  18.703265
1      0.057415  0.037591       3        1  14.893846
3      0.050693  0.005024       4        1  11.904560
8     -0.066166 -0.010279       5        1   9.782760
6     -0.105826  0.058643       6        1   7.660098
0     -0.065259  0.070116       7        1   7.394636
4      0.075582 -0.030039       8        1   5.708305
9     -0.027708  0.042562       9        1   3.371343
5     -0.047627  0.014407      10        1   1.780201, topic_info=       Term       Freq      Total Category  logprob  loglift
152    year  15.000000  15.000000  Default  30.0000  30.0000
194   world   7.000000   7.000000  Default  29.0000  29.0000
28     amzn  24.000000  24.000000  Default  28.0000  28.0000
208  amazon  15.000000  15.000000  Default  27.0000  27.0000
137    goog   7.000000   7.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
0      aapl   0.273700  34.560831  Topic10  -4.4519  -0.8100
7      http   0.087576  60.127202  Topic10  -5.5914  -2.5033
28     amzn   0.031599  24.021458  Topic10  -6.6108  -2.6051
1     bitly   0.031593  11.132648  Topic10  -6.6110  -1.8363
60     tsla   0.031593  15.095509  Topic10  -6.6110  -2.1408

[510 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.115738   aapl
0         2  0.260410   aapl
0         3  0.057869   aapl
0         4  0.144672   aapl
0         5  0.144672   aapl
...     ...       ...    ...
152       8  0.064793   year
283       6  0.785134   yelp
284       6  0.785145   yhoo
254       6  0.785143     yr
162       5  0.719789  ztvgm

[515 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 2, 4, 9, 7, 1, 5, 10, 6])